In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
!pip install lexical-diversity
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
%cd /content/drive/MyDrive/CSNLP/Slang-Detection

/content/drive/MyDrive/CSNLP/Slang-Detection


In [79]:
file_dir = 'data/succeed.csv'
succeed = pd.read_csv(file_dir, index_col = 0)
print(len(succeed))
succeed.columns

3197


Index(['trigger', 'length', 'word', 'generate'], dtype='object')

In [80]:
import pandas as pd
from lexical_diversity import lex_div as ld
from sentence_transformers import SentenceTransformer, util
import math

model = SentenceTransformer('all-MiniLM-L6-v2')

df_new = pd.DataFrame(columns = ['word', 'example'])

for i in range(len(succeed)):
  if i%100 == 0:
    print(i)
  row = succeed.loc[i]
  generated = row['generate']
  if type(generated)!=float:

    sentences = generated.strip('][').split(', ')
    sentences = list(set(sentences))

    #Compute embeddings
    embeddings = model.encode(sentences, convert_to_tensor=True)
    #Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(embeddings, embeddings)
    #Find the pairs with the highest cosine similarity scores
    delete_index = []
    for i in range(len(cosine_scores)-1):
        for j in range(i+1, len(cosine_scores)):
            if cosine_scores[i][j] >= 0.95:
              delete_index.append(j)
    delete_index = list(set(delete_index))

    list_sentences = [i for j, i in enumerate(sentences) if j not in delete_index]

    df_temp = pd.DataFrame(columns = ['word', 'example'])
    df_temp['example'] = list_sentences
    df_temp['word'] = row['word']
    df_new = pd.concat([df_new, df_temp], axis = 0)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [82]:
df_new.to_csv('data/aug_filter_sim.csv')

In [83]:
len(pd.read_csv('data/aug_filter_sim.csv'))

26372